In [1]:
import sagemaker
import boto3 
from sagemaker.image_uris import retrieve
from sagemaker.pytorch import PyTorch, PyTorchModel
import os 
from PIL import Image
from torchvision import transforms
import json

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/dev/.config/sagemaker/config.yaml


In [2]:
sm_boto3 = boto3.client("sagemaker")

sess = sagemaker.Session()

region = sess.boto_session.region_name

In [7]:
# train_data = {'train': 's3://pitahaya/Augmented Dataset/'}
# model_output = 's3://pitahaya/output/'
# model_artifact='s3://pitahaya/output/pytorch-training-2024-11-19-05-56-45-204/output/model.tar.gz'

train_data = os.getenv("TRAINING_PATH")
model_output = os.getenv("OUTPUT_PATH")
model_artifact = os.getenv("ARTIFACT")
role_arn = os.getenv("ROLE_ARN")

In [14]:
train_data, model_artifact, model_output, role_arn

("train': 's3://pitahaya/Augmented Dataset/",
 's3://pitahaya/output/pytorch-training-2024-11-19-05-56-45-204/output/model.tar.gz',
 's3://pitahaya/output/',
 'arn:aws:iam::985570999006:role/service-role/AmazonSageMaker-ExecutionRole-20240718T155914')

In [17]:

if not train_data and model_output and model_artifact and role_arn:
    
    raise ValueError("Las variables de entorno de SAGEMAKER no estan configuradas.")

else: print("rol cargado exitosamente")

rol cargado exitosamente


In [24]:
hyperparameters={
        'epochs': 10,
        'batch_size': 32,
        'learning_rate': 0.001
    }

In [28]:
estimator = PyTorch(
    entry_point='train.py',  
    role=role_arn,
    instance_count=1,  
    instance_type='ml.m5.large', 
    framework_version='1.9.0', 
    py_version='py38',  
    hyperparameters=hyperparameters,
    # model_data=model_artifact,
    output_path=model_output,  
    sagemaker_session=sess
)

In [11]:

# estimator.fit(train_data)
estimator.fit(train_data)


INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: pytorch-training-2024-11-19-05-56-45-204


2024-11-19 05:56:49 Starting - Starting the training job...
2024-11-19 05:57:04 Starting - Preparing the instances for training...
2024-11-19 05:57:33 Downloading - Downloading input data......
2024-11-19 05:58:44 Downloading - Downloading the training image...
2024-11-19 05:59:20 Training - Training image download completed. Training in progress..bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2024-11-19 05:59:23,758 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2024-11-19 05:59:23,761 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-11-19 05:59:23,770 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2024-11-19 05:59:23,774 sagemaker_pytorch_container.training INFO     Invoking user training script.
2024-11-19 05:59:23,935 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus instal

In [7]:
model = PyTorchModel(
    model_data=model_artifact,
    role=role_arn,
    framework_version='1.13',
    py_version='py39',  # Cambia esto si usaste otra versión
)

In [8]:
# Desplegar el modelo
predictor = model.deploy(
    initial_instance_count=1,
    instance_type='ml.m4.xlarge',  # Tipo de instancia para inferencias
)


--------!

In [9]:
endpoint_name = "pytorch-inference-2024-11-19-15-07-51-686"  # Reemplaza con el nombre de tu endpoint

# Crear cliente de SageMaker Runtime
sagemaker_runtime = boto3.client('sagemaker-runtime')

In [12]:
image_path = 'Augmented_Dataset/normal/Fresh_Dragon_Augmented_Data0003.jpg'
image = Image.open(image_path)
image = image.resize((224, 224))  


In [14]:
transform = transforms.Compose([
    transforms.ToTensor(),  # Convertir a tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalización estándar
])


In [15]:
input_tensor = transform(image).unsqueeze(0)  # Añadir dimensión de batch
# Convertir tensor a JSON para enviarlo al endpoint
payload = json.dumps(input_tensor.numpy().tolist())

In [16]:
# Hacer la predicción llamando al endpoint
response = sagemaker_runtime.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType="application/json",  # Tipo de contenido del payload
    Body=payload
)

KeyboardInterrupt: 

In [ ]:
result = json.loads(response['Body'].read().decode())
print(f"Resultado de la predicción: {result}")

In [17]:
predictor.delete_endpoint()